# Wyjątki oraz operacje na plikach

## Wyjątki

Wyjątki są mechanizmem występującym w wielu językach programowani. Służą one do obsługi sytuacji nietypowych, w szczególności sytuacji błędnych. Źródło błędu może być różne:
* błąd w kodzie programu, np. odwołanie do niezainicjowanej zmiennej,
* wprowadzenie błędynych danych przez użytkownika, np. niepoprawnego adresu URL,
* błąd związany ze środowiskiem pracy, np. zerwanie połączenia internetowego.

Mechanizm wyjątków umożliwia obsługę wszystkich typów błędów w ujednolicony sposób. Dodatkowo, w sytuacji, w której błąd wystąpi, pozwala szybko zorientować się jakie jest źródło problemu (w przeciwieństwie do kompunikatu `Segmentation fault` znanego z języka C). 

Jeśli w wyniku wykonania programu spróbujemy podzielić przez zero, to wystąpi błąd `ZeroDivisionError`:
```ruby
a = 1/0
```

In [1]:
a = 1/0

ZeroDivisionError: divided by 0

Opis błędu składa się z nazwy błędu: `ZeroDivisionError`, komunikatu: `divided by 0` oraz tzw. *stacktrace'a*, czyli ciągu wywołań, które doprowadziły do błędu. Ponieważ Ruby nie jest kompilowany, stacktrace zawiera wskazanie plików i numerów linii, w których występuje wywołanie, które doprowadziło do wystąpienia błędu. Śledząc stacktrace, zwykle można dość szybko zidentyfikować miejsce, które wymaga naprawy.

Jeśli np. próbujemy rozwiązać niebanalny problem polegający na określeniu ile potrzeba trabantów do przewiezienia grupy słoni (a liczba słoni mieszczących się w trabancie podawana jest parametrem programu) możemy napisać następujący kod:
```ruby
def liczba_miejsc_w_trabancie
  4
end

def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  liczba_sloni / liczba_miejsc_w_trabancie
end

def oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
  if liczba_sloni % liczba_miejsc_w_trabancie == 0
    0
  else
    1
  end
end

def oblicz_liczbe_wszystkich_samochodow(liczba_sloni)
  oblicz_liczbe_pelnych_samochodow(liczba_sloni) + oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
end

def ile_potrzeba_samochodow?(liczba_sloni)
  puts "Aby przewieźć #{liczba_sloni} słoni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
end

ile_potrzeba_samochodow?(10)
```

In [2]:
def liczba_miejsc_w_trabancie
  4
end

def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  liczba_sloni / liczba_miejsc_w_trabancie
end

def oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
  if liczba_sloni % liczba_miejsc_w_trabancie == 0
    0
  else
    1
  end
end

def oblicz_liczbe_wszystkich_samochodow(liczba_sloni)
  oblicz_liczbe_pelnych_samochodow(liczba_sloni) + oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
end

def ile_potrzeba_samochodow?(liczba_sloni)
  puts "Aby przewieźć #{liczba_sloni} słoni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
end

ile_potrzeba_samochodow?(10)

Aby przewieźć 10 słoni potrzeba 3 samochodów.


Jeśli teraz ktoś zmieni definicję funkcji `liczba_miejsc_w_trabancie`:
```ruby
# wszystkie trabanty są zajmowane przez myszy, dlatego nie można już w nich umieszczać słoni
def liczba_miejsc_w_trabancie
  0
end
```
i zapyta o liczbę trabantów:
```ruby
ile_potrzeba_samochodow?(10)
```
otrzyma błąd:

In [17]:
def liczba_miejsc_w_trabancie
  0
end

ile_potrzeba_samochodow?(10)

W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy.


Przykład ten jest oczywiście bardzo rozbudowany, ale jeśli przyjrzymy się *stacktrace'owi*, to szybko możemy odnaleźć miejsce, które doprowadziło do wystąpienia błędu - przynajmniej pośrednio.

Możliwość zidentyfikowania źródła błędu jest bardzo cenna, niemniej znacznie częściej będziemy chcieli, aby w takiej sytuacji program nadal działał. Co najwyżej dopuścimy specjalny komunikat dla użytkownika informujący o błędzie. 

Aby zabezpieczyć się przed wyjątkiem otaczamy "kod specjalnej troski" słowami kluczowymi `begin`, `rescue` i `end`:
```ruby
def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  begin
    liczba_sloni / liczba_miejsc_w_trabancie
  rescue ZeroDivisionError
    puts "Nie dziel przez zero..."
    0
  end
end
ile_potrzeba_samochodow?(10)
```  

In [18]:
def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  begin
    liczba_sloni / liczba_miejsc_w_trabancie
  rescue ZeroDivisionError
    puts "Nie dziel przez zero..."
    0
  end
end
ile_potrzeba_samochodow?(10)

Nie dziel przez zero...
W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy.


Istotne pytanie, które pojawia się w tym kontekście dotyczy **miejsca**, w którym ma nastąpić obsługa błędu. Tak jak widzimy w powyższym przykładzie, obsługa wprost w funkcji `oblicz_liczbe_pelnych_samochodow` nie wystarczyła, ponieważ ten sam błąd występuje również w funkcji `obliczb_liczbe_niepelnych_samochodow`. Dlatego zdecydowanie lepszym miejscem obsługi błędu jest funkcja `ile_potrzeba_samochodow?`. 

```ruby
# przywracamy poprzednią definicję tej funkcji
def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  liczba_sloni / liczba_miejsc_w_trabancie
end

# oraz dodajemy obsługę błędu na wyższym poziomie
def ile_potrzeba_samochodow?(liczba_sloni)
  begin
    puts "Aby przewieźć #{liczba_sloni} słowni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
  rescue ZeroDivisionError
    puts "W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy."
  end
end
ile_potrzeba_samochodow?(10)
```

In [19]:
# przywracamy poprzednią definicję tej funkcji
def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  liczba_sloni / liczba_miejsc_w_trabancie
end

# oraz dodajemy obsługę błędu na wyższym poziomie
def ile_potrzeba_samochodow?(liczba_sloni)
  begin
    puts "Aby przewieźć #{liczba_sloni} słowni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
  rescue ZeroDivisionError
    puts "W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy."
  end
end
ile_potrzeba_samochodow?(10)

W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy.


Udało nam się uporać z trabantami, które zostały już zajęte przez myszy. To nie są jednak wszystkie możliwe błędy, które mogą się pojawić. Jakiś mało rozgarnięty programista może np. wywołać funkcję w następujący sposób:
```ruby
ile_potrzeba_samochodow?(nil)
```

In [7]:
ile_potrzeba_samochodow?(nil)

NoMethodError: undefined method `/' for nil:NilClass

Oczywiste jest, że nie potrzeba żadnego samochodu, jeśli w ogóle nie mamy słoni do przewiezienia. Wywołanie powyższego kodu skutkuje jednak innym błędem niż wcześniej: `NoMethodError`. Oznacza on, że obiekt `nil` nie ma metody pozwalającej na jego dzielenie (zarówno przez zero, jak przez dowolną inną wartość). Problem ten rozwiążemy również bezpośrednio w funkcji `ile_potrzeba_samochodow?` dodając kolejną klauzulę `rescue`:
```ruby
def ile_potrzeba_samochodow?(liczba_sloni)
  begin
    puts "Aby przewieźć #{liczba_sloni} słowni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
  rescue ZeroDivisionError
    puts "W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy."
  rescue NoMethodError
    puts "Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '#{liczba_sloni}' typu #{liczba_sloni.class}."
  end
end
ile_potrzeba_samochodow?(nil)
ile_potrzeba_samochodow?("5")
ile_potrzeba_samochodow?(/aaaa/)
```

In [8]:
def ile_potrzeba_samochodow?(liczba_sloni)
  begin
    puts "Aby przewieźć #{liczba_sloni} słowni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
  rescue ZeroDivisionError
    puts "W trabancie nie można umieścić żadnego słownia, bo jest on już zajmowany przez myszy."
  rescue NoMethodError
    puts "Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '#{liczba_sloni}' typu #{liczba_sloni.class}."
  end
end
ile_potrzeba_samochodow?(nil)
ile_potrzeba_samochodow?("5")
ile_potrzeba_samochodow?(/aaaa/)

Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '' typu NilClass.
Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '5' typu String.
Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '(?-mix:aaaa)' typu Regexp.


Wiemy już jak obsługiwać wyjątki. A jak je zgłaszać? Służy do tego polecenie `raise`:
```ruby
raise "Rzucam wyjątek bo tak."
```

In [9]:
raise "Rzucam wyjątek bo tak."

RuntimeError: Rzucam wyjątek bo tak.

Jeśli nie podamy nazwy wyjątku, to domyślnie będzie to błąd `RuntimeError`. Aby rzucić wyjątek określonego typu, musimy użyć konstruktora, który przyjmuje komunikat wyjątki jako parametr wywołania:
```ruby
raise ArgumentError.new("Jakiś dziwny ten argument")
```

In [10]:
raise ArgumentError.new("Jakiś dziwny ten argument")

ArgumentError: Jakiś dziwny ten argument

Błąd tego rodzaju obsługujemy tak jak inne wyjątki:
```ruby
begin
  raise ArgumentError.new("Jakiś dziwny ten argument")
rescue ArgumentError => ex
  puts ex
end
```

In [11]:
begin
  raise ArgumentError.new("Jakiś dziwny ten argument")
rescue ArgumentError => ex
  puts ex
end

Jakiś dziwny ten argument


W powyższym kodzie została zostosowana dodatkowa możliwość przy obsłudze wyjątków - przechwycenie ich do zmiennej, w tym wypadku `ex`. Zmienna taka zawiera wszystkie istotne informacje dotyczące wyjątku, m.in. komunikat i stos wywołań:
```ruby
begin
  raise ArgumentError.new("Jakiś dziwny ten argument")
rescue ArgumentError => ex
  puts ex
  puts ex.backtrace[0..5]
end
```

In [12]:
begin
  raise ArgumentError.new("Jakiś dziwny ten argument")
rescue ArgumentError => ex
  puts ex
  puts ex.backtrace[0..5]
end

Jakiś dziwny ten argument
["<main>:1:in `<main>'", "/usr/local/rvm/gems/ruby-2.0.0-p0/gems/iruby-0.1.13/lib/iruby/backend.rb:8:in `eval'", "/usr/local/rvm/gems/ruby-2.0.0-p0/gems/iruby-0.1.13/lib/iruby/backend.rb:8:in `eval'", "/usr/local/rvm/gems/ruby-2.0.0-p0/gems/iruby-0.1.13/lib/iruby/kernel.rb:110:in `execute_request'", "/usr/local/rvm/gems/ruby-2.0.0-p0/gems/iruby-0.1.13/lib/iruby/kernel.rb:62:in `run'", "/usr/local/rvm/gems/ruby-2.0.0-p0/gems/iruby-0.1.13/lib/iruby/command.rb:30:in `run_kernel'"]


### Zadanie 1

Zmodyfikuj funkcje `oblicz_liczbe_pelnych_samochodow` oraz `oblicz_liczbe_niepelnych_samochodow` tak by rzucały wyjątek `ArgumentError`, jeśli liczba słoni jest mniejsza od 0. Następnie zmodyfikuj funkcję `ile_potrzeba_samochodow?`, tak by obsługiwała ten wyjątek. W następnej kolejności zmodyfikuj funkcję `liczba_miejsc_w_trabancie`, tak by ponownie można było wozić słonie. Przetestuj główną funkcję przekazując jej różne parametry (również mało oczywiste). Czy tak zdefiniowana funkcja jest niezawodna?

In [42]:
def liczba_miejsc_w_trabancie
  4
end

def oblicz_liczbe_pelnych_samochodow(liczba_sloni)
  begin
   if liczba_sloni < 0
    raise ArgumentError.new("Za mało słoni!")
   end
        liczba_sloni / liczba_miejsc_w_trabancie
  end
end

def oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
  begin
    if liczba_sloni % liczba_miejsc_w_trabancie == 0
      0
      if liczba_sloni < 0
        raise ArgumentError.new("Liczba jest mniejsza od 0!") 
      end
      else
      1
    end
    
 # rescue ArgumentError
  #  puts "Liczba słoni jest mniejsza od 0!"
  end
end

def oblicz_liczbe_wszystkich_samochodow(liczba_sloni)
  oblicz_liczbe_pelnych_samochodow(liczba_sloni) + oblicz_liczbe_niepelnych_samochodow(liczba_sloni)
end


def ile_potrzeba_samochodow?(liczba_sloni)
  begin
    puts "Aby przewieźć #{liczba_sloni} słoni potrzeba #{oblicz_liczbe_wszystkich_samochodow(liczba_sloni)} samochodów."
  rescue ZeroDivisionError
    puts "W trabancie nie można umieścić żadnego słonia, bo jest on już zajmowany przez myszy."
  rescue NoMethodError
    puts "Trzeba podać liczbę słoni a nie jakąś dziwną wartość: '#{liczba_sloni}' typu #{liczba_sloni.class}."
    rescue ArgumentError
      puts "Liczba słoni jest mniejsza niż 0!"
  end
end

ile_potrzeba_samochodow?(109)

Aby przewieźć 109 słoni potrzeba 28 samochodów.


## Obsługa plików

Dwie podstawowe operacje na plikach to odczytywanie i zapisywanie danych do pliku. 

Obie operacje wymagają aby plik został najpierw *otwarty*. Służy do tego metoda `File.open`, która wymaga podania ścieżki do pliku:
```ruby
plik = File.open("data/authors.csv")
puts plik.readline
plik.close
```

In [43]:
plik = File.open("data/authors.csv")
puts plik.readline
plik.close

Orson_Scott_Card,1951-08-24,,http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300



Po otwarciu pliku, można na nim wykonywać operacje. W powyższym przykładzie metoda `readline` wczytała pierwszą linię pliku. Po jej wyświetleniu plik został zamknięty poleceniem `close`.

Ponieważ po otwarciu pliku trzeba go zawsze zamknąć (choć programiści i programistki często o tym zapominają), Ruby oferuje alternatywną składnie do obsługi plików:
```ruby
File.open("data/authors.csv") do |plik|
  puts plik.readline
end
```

In [44]:
File.open("data/authors.csv") do |plik|
  puts plik.readline
end

Orson_Scott_Card,1951-08-24,,http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300



Jej przewaga nad poprzednią metodą jest taka, że po opuszczeniu bloku `do` ... `end` plik zostanie automatycznie zamknięty. A jeśli zapomnielibyśmy słowa `end`, to program się po prostu nie wykona (wystąpi błąd składni).

Wyświetlenie 10 pierwszych wierszy wygląda następująco:
```ruby
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline
  end
end
```

In [45]:
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline
  end
end

Orson_Scott_Card,1951-08-24,,http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300

Andrzej_Sapkowski,1948-06-21,,http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300

Jack_London,1876-01-12,1916-11-22,http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300

Henning_Mankell,1948-02-03,,http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300

Stanisław_Lem,1921-09-12,2006-03-27,http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300

Antoni_Gołubiew,1907-02-25,1979-06-27,http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300

Mikołaj_Kopernik,1473-02-19,1543-05-24,http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300

Maria_Franciszka_Kozłowska,1862-05-27,,http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300

George_Orwell,1903-06

10

Ponieważ linie w pliku zakończone są znakiem przejścia do nowej linii, warto pamiętać o zastosowaniu funkcji `chomp` przy wyświetlaniu poszczególnych wierszy:
```ruby
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline.chomp
  end
end
```

In [46]:
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline.chomp
  end
end

Orson_Scott_Card,1951-08-24,,http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300
Andrzej_Sapkowski,1948-06-21,,http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300
Jack_London,1876-01-12,1916-11-22,http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300
Henning_Mankell,1948-02-03,,http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300
Stanisław_Lem,1921-09-12,2006-03-27,http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300
Antoni_Gołubiew,1907-02-25,1979-06-27,http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300
Mikołaj_Kopernik,1473-02-19,1543-05-24,http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300
Maria_Franciszka_Kozłowska,1862-05-27,,http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300
George_Orwell,1903-06-25,1950

10

Poszczególne wiersze wczytywane za pomocą polecenia `readline` są po prostu łańcuchami znaków. Jeśli chcemy np. wyświetlić pierwsze pole w pliku, w którym stosuje się przecinek do oddzielenia pól (format CSV), to możemy zrobić to następująco:
```ruby
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline.chomp.split(",").first
  end
end
```

In [47]:
File.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.readline.chomp.split(",").first
  end
end

Orson_Scott_Card
Andrzej_Sapkowski
Jack_London
Henning_Mankell
Stanisław_Lem
Antoni_Gołubiew
Mikołaj_Kopernik
Maria_Franciszka_Kozłowska
George_Orwell
William_Szekspir


10

Do obsługi tego formatu istnieje osobna klasa `CSV`, która np. bierzę pod uwagę, to, że przecinek może również być treścią pola. Ponadto obsługa plików `csv` za pomocą tej klasy jest znacznie szybsza. Klasa ta nie jest jednak domyślnie dostępna, dlatego trzeba załadować ją za pomocą polecenia `require`. Ponadto wczytywanie kolejnych wierszy odbywa się za pomocą metody `shift`:
```ruby
require 'csv'
CSV.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.shift.first
  end
end
```

In [48]:
require 'csv'
CSV.open("data/authors.csv") do |plik|
  10.times do 
    puts plik.shift.first
  end
end

Orson_Scott_Card
Andrzej_Sapkowski
Jack_London
Henning_Mankell
Stanisław_Lem
Antoni_Gołubiew
Mikołaj_Kopernik
Maria_Franciszka_Kozłowska
George_Orwell
William_Szekspir


10

Zapisywanie do pliku jest bardzo podobne do wyświetlania treści na ekranie. Zapis do kolejnych linii realizowany jest za pomocą polecenia `puts`. Jednakże próba zapisu do nieistniejącego pliku kończy się porażką:
```ruby
File.open("data/moj_plik.txt") do |plik|
  1.upto(10) do |indeks|
    plik.puts "To jest #{indeks} linia"
  end
end
```

In [49]:
File.open("data/moj_plik.txt") do |plik|
  1.upto(10) do |indeks|
    plik.puts "To jest #{indeks} linia"
  end
end

Errno::ENOENT: No such file or directory - data/moj_plik.txt

W powyższym wywołaniu wystąpił wyjątek `Errno::ENOENT`, oznaczający, że plik, który chcieliśmy otworzyć nie isnieje. Aby rozwiązać ten problem, konieczne jest zmodyfikowanie wywołania funkcji `open`, przez dodanie flagi "w" informującej, że plik otwierany jest w trybie *do zapisu*:
```ruby
File.open("data/moj_plik.txt","w") do |plik|
  1.upto(10) do |indeks|
    plik.puts "To jest #{indeks} linia"
  end
end
```

In [50]:
File.open("data/moj_plik.txt","w") do |plik|
  1.upto(10) do |indeks|
    plik.puts "To jest #{indeks} linia"
  end
end

1

To że kod faktycznie zadziałał można zweryfikować zaglądając do pliku `data/moj_plik.txt`.

### Zadanie 2

Napisz program, który przepisze wszystkie pierwsze pola (czyli nazwiska autorów) z pliku `data/authors.csv` do pliku `data/names.txt`. W nazwiskach znaki podkreślenia mają zostać zastąpione spacjami. Aby odwiedzić wszystkie wiersze w pliku użyj metody `each`. Zwróć uwagę, że plik do którego zapisujesz i plik który odczytujesz muszą być dostępne za pomocą innych zmiennych (przujmując, że pracujemy na obu plikach jednocześnie) oraz, że w pliku `authors.csv` jest więcej pozycji niż 10.

In [4]:
require 'csv'
CSV.open("data/authors.csv") do |plik_authors|
  File.open("data/names.txt","w") do |plik_names|
    plik_authors.each do |linijka|
      plik_names.puts linijka[0].gsub(/_/, ' ')
    end
  end
end

### Zadanie 3

Wykonaj to samo zadanie co w poprzednim ćwiczeniu, uwzględniając następujące różnice:
* dane mają trafić do pliku `data/sorted_names.txt`,
* dane mają być posortowane zgodnie z porządkiem alfabetycznym.

Możesz pomiąć fakt, że w pliku na pierwszym miejscu pojawia się imię, a nie nazwisko autora.

In [7]:
require 'csv'
CSV.open("data/authors.csv") do |infile|
  File.open("data/sorted_names.txt", "w") do |outfile|
    infile.map{|elem| elem.first.gsub(/_/, ' ')}.sort.each{|a| outfile.print a + "\n"}
  end
end

["Adam Mickiewicz", "Adam Zelga", "Adolf Hitler", "Aldous Huxley", "Aleksander Dumas (ojciec)", "Aleksander Majkowski", "Alfred Szklarski", "Andrzej Pilipiuk", "Andrzej Sapkowski", "Anne Rice", "Antoni Gołubiew", "Benedykt Chmielowski", "Boris Akunin", "Christopher Tolkien", "Clive Staples Lewis", "Dorota Terakowska", "Eliza Orzeszkowa", "Franz Kafka", "Gabriela Zapolska", "Gene Wolfe", "George Orwell", "George R. R. Martin", "Henning Mankell", "Henryk Sienkiewicz", "Honoré de Balzac", "Howard Phillips Lovecraft", "Ian Fleming", "Ilja Ilf", "Isaac Asimov", "Iwan Turgieniew", "J. K. Rowling", "Jack London", "Jan Kochanowski", "Jan Paweł II", "Janusz Zajdel", "Jeffery Deaver", "Jewgienij Pietrow (pisarz)", "John Ronald Reuel Tolkien", "Karol Olgierd Borchardt", "Lew Tołstoj", "Lucy Maud Montgomery", "Maria Franciszka Kozłowska", "Mikołaj Kopernik", "Orson Scott Card", "Paulo Coelho", "Rafał A. Ziemkiewicz", "Ryszard Kapuściński", "Stanisław Lem", "Tadeusz Dołęga-Mostowicz", "Terry Pratch